In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data as data
from PIL import Image

MLP,LeNet
=========

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(28*28*3,8192)
        self.BN1 = nn.BatchNorm1d(8192)
        self.fc2 = nn.Linear(8192,8192)
        self.BN2 = nn.BatchNorm1d(8192)
        self.fc3 = nn.Linear(8192,4096)
        self.BN3 = nn.BatchNorm1d(4096)
        self.fc4 = nn.Linear(4096,2048)
        self.BN4 = nn.BatchNorm1d(2048)
        self.fc5 = nn.Linear(2048,18)
        
    def forward(self,x):
        out = x.view(-1,28*28*3)
#         out = F.sigmoid(self.fc1(out))
#         out = F.sigmoid(self.fc2(out))
#         out = F.sigmoid(self.fc3(out))
#         out = F.sigmoid(self.fc4(out))

        out = F.sigmoid(self.BN1(self.fc1(out)))
        out = F.sigmoid(self.BN2(self.fc2(out)))
        out = F.sigmoid(self.BN3(self.fc3(out)))
        out = F.sigmoid(self.BN4(self.fc4(out)))
        out = F.sigmoid(self.fc5(out))
        return out

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 18)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0),-1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
        

In [4]:
for learning_rate in [0.1,0.05,0.01,0.005,0.001,0.0005,0.0001]:
    initial_learning_rate = learning_rate
    for num_epochs in [100,150,200,250]:
#====================================================================================================
# MLP
#====================================================================================================
        mlp = MLP()
        mlp.cuda()


        batch_size = 100
        learning_rate=initial_learning_rate
        img_dir = "/home/junhyun/Junhyun/train/"

        # img_data = dset.ImageFolder(img_dir)
        img_data_mlp = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.778163803690477, 0.62366406713856704, 0.62748488269742386],std = [0.18654390350275066, 0.25577185166630317, 0.22957029180170951])
        ]))

        img_batch_mlp = data.DataLoader(img_data_mlp, batch_size=batch_size,
                                    shuffle=True, num_workers=2)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            i=0
            if (epoch > 100)and(epoch%20==0):
                learning_rate= learning_rate*0.2
                print(learning_rate)

            for img,label in img_batch_mlp:
                images = Variable(img).cuda()
                labels = Variable(label).cuda()

                # Forward + Backward + Optimize
                optimizer.zero_grad()
                outputs = mlp(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                i+=1
                if i%100 ==0:
                    print('step : %d'%i)

            if epoch%10 ==0:
                print ('Epoch [%d/%d], Loss: %.9f'%(epoch+1, num_epochs, loss.data[0]))


        img_dir = "/home/junhyun/Junhyun/val"

        test_dataset = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.778163803690477, 0.62366406713856704, 0.62748488269742386],std = [0.18654390350275066, 0.25577185166630317, 0.22957029180170951])
        ]))


        test_batch = data.DataLoader(test_dataset, batch_size=100,
                                    shuffle=True, num_workers=2)

        # Test the Model\
        mlp.eval()
        correct = 0
        total = 0

        for img,labels in test_batch:
            images = Variable(img).cuda()
            outputs = mlp(images)
            _, predicted = torch.max(outputs.data,1)
            total+= labels.size(0)
            correct += (predicted.cpu() == labels).sum()

        Acc = round(100.0 * float(correct) / float(total),2)
        print('Accuracy of the network on the test images: %f %%' % (Acc))





        save_name = 'mlp_Acc' + str(Acc)[:5] + '_epoch' + str(num_epochs) + '_lr' + str(initial_learning_rate) + '.pkl'
        print(save_name)
        torch.save(mlp.state_dict(), save_name)



#====================================================================================================
# MLP
#====================================================================================================


        lenet = LeNet()
        lenet.cuda()


        batch_size = 100
        learning_rate=initial_learning_rate

        img_dir = "/home/junhyun/Junhyun/train/"

        img_data_lenet = dset.ImageFolder(img_dir, transforms.Compose([
        #     transforms.ToPILImage(),
            transforms.Scale(32,interpolation=Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.778163803690477, 0.62366406713856704, 0.62748488269742386],std = [0.18654390350275066, 0.25577185166630317, 0.22957029180170951])
        ]))

        img_batch_lenet = data.DataLoader(img_data_lenet, batch_size=batch_size,
                                    shuffle=True, num_workers=2)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(lenet.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            i=0
            if (epoch > 100)and(epoch%20==0):
                learning_rate= learning_rate*0.2
                print(learning_rate)
            for img,label in img_batch_lenet:
                images = Variable(img).cuda()
                labels = Variable(label).cuda()

                # Forward + Backward + Optimize
                optimizer.zero_grad()
                outputs = lenet(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                i+=1
                if i%100 ==0:
                    print('step : %d'%i)

            if epoch%10 ==0:
                print ('Epoch [%d/%d], Loss: %.9f'%(epoch+1, num_epochs, loss.data[0]))


        img_dir = "/home/junhyun/Junhyun/val"

        test_dataset = dset.ImageFolder(img_dir, transforms.Compose([
        #     transforms.ToPILImage(),
            transforms.Scale(32,interpolation=Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.778163803690477, 0.62366406713856704, 0.62748488269742386],std = [0.18654390350275066, 0.25577185166630317, 0.22957029180170951])
        ]))

        test_batch = data.DataLoader(test_dataset, batch_size=batch_size,
                                    shuffle=False, num_workers=2)

        # Test the Model
        lenet.eval()
        correct = 0
        total = 0

        for img,labels in test_batch:
            images = Variable(img).cuda()
            outputs = lenet(images)
            _, predicted = torch.max(outputs.data,1)
            total+= labels.size(0)
            correct += (predicted.cpu() == labels).sum()


        Acc = round(100.0 * float(correct) / float(total),2)
        print('Accuracy of the network on the test images: %f %%' % (Acc))





        save_name = 'lenet_Acc' + str(Acc)[:5] + '_epoch' + str(num_epochs) + '_lr' + str(initial_learning_rate) +  '.pkl'
        print(save_name)

        torch.save(lenet.state_dict(), save_name)

Epoch [1/100], Loss: 2.880234718
Epoch [11/100], Loss: 2.871831417
Epoch [21/100], Loss: 2.930654526
Epoch [31/100], Loss: 2.955864668
Epoch [41/100], Loss: 2.804604292
Epoch [51/100], Loss: 2.880234718
Epoch [61/100], Loss: 2.930654526
Epoch [71/100], Loss: 2.813007832
Epoch [81/100], Loss: 2.787797689
Epoch [91/100], Loss: 2.863427877
Accuracy of the network on the test images: 6.350000 %
mlp_Acc6.35_epoch100_lr0.1.pkl
Epoch [1/100], Loss: 6.978487968
Epoch [11/100], Loss: 2.748284578
Epoch [21/100], Loss: 2.816827536
Epoch [31/100], Loss: 2.851258278
Epoch [41/100], Loss: 2.776464939
Epoch [51/100], Loss: 2.821700573
Epoch [61/100], Loss: 2.846170425
Epoch [71/100], Loss: 2.763332367
Epoch [81/100], Loss: 2.881966352
Epoch [91/100], Loss: 2.793083668
Accuracy of the network on the test images: 10.770000 %
lenet_Acc10.77_epoch100_lr0.1.pkl
Epoch [1/150], Loss: 2.847507954
Epoch [11/150], Loss: 2.847507954
Epoch [21/150], Loss: 2.813894510
Epoch [31/150], Loss: 2.906331539
Epoch [41/1

Epoch [131/200], Loss: 2.897041321
0.002
Epoch [141/200], Loss: 2.855024576
Epoch [151/200], Loss: 2.863427877
0.0004
Epoch [161/200], Loss: 2.880234718
Epoch [171/200], Loss: 2.855024576
8e-05
Epoch [181/200], Loss: 2.855024576
Epoch [191/200], Loss: 2.813007832
Accuracy of the network on the test images: 8.080000 %
mlp_Acc8.08_epoch200_lr0.05.pkl
Epoch [1/200], Loss: 2.767172337
Epoch [11/200], Loss: 1.791464329
Epoch [21/200], Loss: 1.957343102
Epoch [31/200], Loss: 1.539062977
Epoch [41/200], Loss: 1.613857269
Epoch [51/200], Loss: 1.356849790
Epoch [61/200], Loss: 1.563727617
Epoch [71/200], Loss: 1.177800655
Epoch [81/200], Loss: 2.429996967
Epoch [91/200], Loss: 2.125413418
Epoch [101/200], Loss: 2.090378284
Epoch [111/200], Loss: 2.134461641
0.01
Epoch [121/200], Loss: 1.841791391
Epoch [131/200], Loss: 2.200787783
0.002
Epoch [141/200], Loss: 1.914367795
Epoch [151/200], Loss: 1.905696034
0.0004
Epoch [161/200], Loss: 1.819542050
Epoch [171/200], Loss: 1.817196012
8e-05
Epoch 

Epoch [101/250], Loss: 0.079744756
Epoch [111/250], Loss: 0.047181688
0.002
Epoch [121/250], Loss: 0.062666774
Epoch [131/250], Loss: 0.022972628
0.0004
Epoch [141/250], Loss: 0.119502030
Epoch [151/250], Loss: 0.099486277
8e-05
Epoch [161/250], Loss: 0.116957657
Epoch [171/250], Loss: 0.102685891
1.6e-05
Epoch [181/250], Loss: 0.190431774
Epoch [191/250], Loss: 0.037634794
3.2e-06
Epoch [201/250], Loss: 0.023487682
Epoch [211/250], Loss: 0.144931704
6.4e-07
Epoch [221/250], Loss: 0.134416789
Epoch [231/250], Loss: 0.174610063
1.28e-07
Epoch [241/250], Loss: 0.035082988
Accuracy of the network on the test images: 64.040000 %
lenet_Acc64.04_epoch250_lr0.01.pkl
Epoch [1/100], Loss: 2.812580824
Epoch [11/100], Loss: 2.804376602
Epoch [21/100], Loss: 2.755668879
Epoch [31/100], Loss: 2.737233162
Epoch [41/100], Loss: 2.770273447
Epoch [51/100], Loss: 2.778266191
Epoch [61/100], Loss: 2.713980436
Epoch [71/100], Loss: 2.740723610
Epoch [81/100], Loss: 2.748883963
Epoch [91/100], Loss: 2.723

Epoch [111/150], Loss: 0.034896195
0.0002
Epoch [121/150], Loss: 0.013107740
Epoch [131/150], Loss: 0.007562830
4e-05
Epoch [141/150], Loss: 0.003549383
Accuracy of the network on the test images: 66.920000 %
lenet_Acc66.92_epoch150_lr0.001.pkl
Epoch [1/200], Loss: 2.608554125
Epoch [11/200], Loss: 2.308681726
Epoch [21/200], Loss: 2.203678370
Epoch [31/200], Loss: 2.153811932
Epoch [41/200], Loss: 2.176387310
Epoch [51/200], Loss: 2.124035120
Epoch [61/200], Loss: 2.185572147
Epoch [71/200], Loss: 2.101729393
Epoch [81/200], Loss: 2.113082170
Epoch [91/200], Loss: 2.071791887
Epoch [101/200], Loss: 2.114568233
Epoch [111/200], Loss: 2.069540024
0.0002
Epoch [121/200], Loss: 2.132474184
Epoch [131/200], Loss: 2.079413652
4e-05
Epoch [141/200], Loss: 2.051523447
Epoch [151/200], Loss: 2.088352203
8e-06
Epoch [161/200], Loss: 2.085832119
Epoch [171/200], Loss: 2.055813789
1.6e-06
Epoch [181/200], Loss: 2.048366547
Epoch [191/200], Loss: 2.063324451
Accuracy of the network on the test ima

8e-07
Epoch [181/250], Loss: 2.056764126
Epoch [191/250], Loss: 2.030493259
1.6e-07
Epoch [201/250], Loss: 2.040055513
Epoch [211/250], Loss: 2.056095362
3.2e-08
Epoch [221/250], Loss: 2.013525486
Epoch [231/250], Loss: 2.013065338
6.4e-09
Epoch [241/250], Loss: 2.016839266
Accuracy of the network on the test images: 50.190000 %
mlp_Acc50.19_epoch250_lr0.0005.pkl
Epoch [1/250], Loss: 2.771508455
Epoch [11/250], Loss: 1.357573152
Epoch [21/250], Loss: 1.106738806
Epoch [31/250], Loss: 0.898245931
Epoch [41/250], Loss: 0.857666373
Epoch [51/250], Loss: 0.597440839
Epoch [61/250], Loss: 0.547264040
Epoch [71/250], Loss: 0.348891765
Epoch [81/250], Loss: 0.276636839
Epoch [91/250], Loss: 0.180542409
Epoch [101/250], Loss: 0.117749304
Epoch [111/250], Loss: 0.087481923
0.0001
Epoch [121/250], Loss: 0.066063389
Epoch [131/250], Loss: 0.056193393
2e-05
Epoch [141/250], Loss: 0.027760522
Epoch [151/250], Loss: 0.019557392
4e-06
Epoch [161/250], Loss: 0.014029599
Epoch [171/250], Loss: 0.008256